# Strompreise

In [1]:
import time
from typing import Optional, Tuple, List
import datetime as dt
import pandas as pd
import pytz
import requests
import streamlit as st
import plotly.graph_objects as go

In [5]:
resolution_choice = "quarterly"

In [9]:
# ---------------------------------------------------------
# SMARD Loader
# ---------------------------------------------------------
SERIES_ID = "4169"
REGION_CANDIDATES = ["DE-LU", "DE"]
SMARD_BASE = "https://www.smard.de/app"
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; Streamlit-SMARD/1.0)"}
_last_tried: List[str] = []

class SmardError(Exception):
    pass

def _safe_get_json(url: str, timeout: int = 30) -> dict:
    _last_tried.append(url)
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    if r.status_code != 200:
        raise SmardError(f"HTTP {r.status_code} für {url}")
    try:
        return r.json()
    except Exception:
        snippet = (r.text or "")[:160].replace("\n", " ")
        ctype = r.headers.get("Content-Type", "")
        raise SmardError(f"Kein JSON von {url}. Content-Type='{ctype}', Antwort: '{snippet}...'")

def _get_index(region: str, resolution: str) -> list[int]:
    url = f"{SMARD_BASE}/chart_data/{SERIES_ID}/{region}/index_{resolution}.json"
    data = _safe_get_json(url)
    ts = data.get("timestamps") or []
    if not ts:
        raise SmardError(f"Keine timestamps in index_{resolution}.json ({region})")
    return ts

def _try_load_series(region: str, resolution: str, ts: int) -> Optional[pd.DataFrame]:
    for path in ["table_data", "chart_data"]:
        url = f"{SMARD_BASE}/{path}/{SERIES_ID}/{region}/{SERIES_ID}_{region}_{resolution}_{ts}.json"
        try:
            data = _safe_get_json(url)
            series = data.get("series")
            if series:
                return pd.DataFrame(series, columns=["ts_ms", "eur_per_mwh"])
        except SmardError:
            continue
    return None

# @st.cache_data(ttl=900)
def load_smard_series(prefer_resolution: str = "quarterhour", max_backsteps: int = 12) -> Tuple[pd.DataFrame, str, str]:
    resolutions = [prefer_resolution] + ([r for r in ["hour"] if r != prefer_resolution])
    for region in REGION_CANDIDATES:
        for resolution in resolutions:
            try:
                idx = _get_index(region, resolution)
            except SmardError:
                continue
            for ts in reversed(idx[-(max_backsteps + 1):]):
                df = _try_load_series(region, resolution, ts)
                if df is not None and not df.empty:
                    return df, resolution, region
                time.sleep(0.15)
    raise SmardError("Keine gültige SMARD-Datei gefunden (region/auflösung/ts).")

In [10]:
df_raw, used_resolution, used_region = load_smard_series(prefer_resolution=resolution_choice, max_backsteps=12)

In [11]:
df_raw

,ts_ms,eur_per_mwh
0,1760306400000,93.45
1,1760310000000,90.26
2,1760313600000,90.83
3,1760317200000,89.12
4,1760320800000,93.45
...,...,...
163,1760893200000,NaN
164,1760896800000,NaN
165,1760900400000,NaN
166,1760904000000,NaN


In [12]:
# ---------------------------------------------------------
# Data preparation
# ---------------------------------------------------------
tz_berlin = pytz.timezone("Europe/Berlin")
df_raw["ts"] = pd.to_datetime(df_raw["ts_ms"], unit="ms", utc=True).dt.tz_convert("Europe/Berlin")
df_raw["ct_per_kwh"] = df_raw["eur_per_mwh"] * 0.1

# ---------------------------------------------------------
# Zeitfenster: von aktuellem Mittag (12:00) bis nächstes Mittag (12:00)
# ---------------------------------------------------------
tz_berlin = pytz.timezone("Europe/Berlin")
now = dt.datetime.now(tz=tz_berlin)
today = now.date()

# define nominal window 12:00→12:00
start_window = tz_berlin.localize(dt.datetime.combine(today, dt.time(12, 0)))
end_window = start_window + dt.timedelta(days=1)

df_raw["ts"] = pd.to_datetime(df_raw["ts_ms"], unit="ms", utc=True).dt.tz_convert("Europe/Berlin")
df_raw["ct_per_kwh"] = df_raw["eur_per_mwh"] * 0.1

# Filter: keep data that falls inside that nominal 24-h window
df = df_raw[(df_raw["ts"] >= start_window - dt.timedelta(hours=12)) &
            (df_raw["ts"] < end_window + dt.timedelta(hours=12))].copy()

# ensure we actually cover the full delivery range; extend end to +12 h after last timestamp if needed
if not df.empty:
    last_ts = df["ts"].max()
    if last_ts < end_window:
        end_window = last_ts + dt.timedelta(hours=12)

# if df.empty:
#     st.info("Für dieses Zeitfenster liegen noch keine Day-Ahead-Daten vor.")
#     st.stop()

# # ---------------------------------------------------------
# # Komponenten: Spot + Gebühren (inkl. MwSt)
# # ---------------------------------------------------------
# fees = st.session_state.fees
# df["spot_ct"] = df["ct_per_kwh"]

# fees_no_vat = (
#     fees["stromsteuer_ct"]
#     + fees["umlagen_ct"]
#     + fees["konzessionsabgabe_ct"]
#     + fees["netzentgelt_ct"]
# )
# df["fees_incl_vat_ct"] = (fees_no_vat + df["spot_ct"]) * (fees["mwst"] / 100.0) + fees_no_vat
# df["total_ct"] = df["spot_ct"] + df["fees_incl_vat_ct"]


In [13]:
df_raw

,ts_ms,eur_per_mwh,ts,ct_per_kwh
0,1760306400000,93.45,2025-10-13 00:00:00+02:00,9.345
1,1760310000000,90.26,2025-10-13 01:00:00+02:00,9.026
2,1760313600000,90.83,2025-10-13 02:00:00+02:00,9.083
3,1760317200000,89.12,2025-10-13 03:00:00+02:00,8.912
4,1760320800000,93.45,2025-10-13 04:00:00+02:00,9.345
...,...,...,...,...
163,1760893200000,NaN,2025-10-19 19:00:00+02:00,NaN
164,1760896800000,NaN,2025-10-19 20:00:00+02:00,NaN
165,1760900400000,NaN,2025-10-19 21:00:00+02:00,NaN
166,1760904000000,NaN,2025-10-19 22:00:00+02:00,NaN
